In [ ]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
import datetime as dt
import urllib.parse
from selenium.webdriver.common.by import By

In [ ]:
from datetime import datetime, timedelta

# 입력한 start, end date 사이의 날짜를 생성해주는 함수 정의
def date_range(start, end):
    start = datetime.strptime(start, "%Y-%m-%d")
    end = datetime.strptime(end, "%Y-%m-%d")
    dates = [(start + timedelta(days=i)).strftime("%Y-%m-%d") for i in range((end-start).days+1)]
    return dates

# 환승연애 시즌2 방영일자 기준으로 날짜 리스트 생성
dates = date_range("2022-07-15", "2022-11-12")
print(dates)

['2022-07-15', '2022-07-16', '2022-07-17', '2022-07-18', '2022-07-19', '2022-07-20', '2022-07-21', '2022-07-22', '2022-07-23', '2022-07-24', '2022-07-25', '2022-07-26', '2022-07-27', '2022-07-28', '2022-07-29', '2022-07-30', '2022-07-31', '2022-08-01', '2022-08-02', '2022-08-03', '2022-08-04', '2022-08-05', '2022-08-06', '2022-08-07', '2022-08-08', '2022-08-09', '2022-08-10', '2022-08-11', '2022-08-12', '2022-08-13', '2022-08-14', '2022-08-15', '2022-08-16', '2022-08-17', '2022-08-18', '2022-08-19', '2022-08-20', '2022-08-21', '2022-08-22', '2022-08-23', '2022-08-24', '2022-08-25', '2022-08-26', '2022-08-27', '2022-08-28', '2022-08-29', '2022-08-30', '2022-08-31', '2022-09-01', '2022-09-02', '2022-09-03', '2022-09-04', '2022-09-05', '2022-09-06', '2022-09-07', '2022-09-08', '2022-09-09', '2022-09-10', '2022-09-11', '2022-09-12', '2022-09-13', '2022-09-14', '2022-09-15', '2022-09-16', '2022-09-17', '2022-09-18', '2022-09-19', '2022-09-20', '2022-09-21', '2022-09-22', '2022-09-23', '2022

In [ ]:
import pandas as pd
# 최종 크롤링 데이터를 담을 빈 데이터 프레임 생성
df = pd.DataFrame(columns=['date', 'tweet'])
n = 0

# 트위터 검색 기능을 활용한 크롤링
# 트윗 검색시 날짜는 최소 2일 이상이어야하므로, 날짜 리스트 내에 담긴 2개 날짜 내의 트윗 수집
for date_id in range(0, len(dates)-1):
    if date_id % 2 == 0:
        startdate=dates[date_id]
        untildate=dates[date_id+1]
        enddate=dates[date_id+1]

        # 서치할 키워드 정의
        keyword = urllib.parse.quote_plus("환승연애")

        # 트위터 서치 쿼리를 담은 URL
        url='https://twitter.com/search?q='+keyword+'%20since%3A'+str(startdate)+'%20until%3A'+str(untildate)+'&src=typed_query&f=live'
        path = './chromedriver' 
        driver = webdriver.Chrome(path)
        driver.get(url)
        time.sleep(3)

        html = driver.page_source
        # 스크롤 높이 가져옴
        last_height = driver.execute_script("return document.body.scrollHeight")

        while True:
            # 끝까지 스크롤 내리기
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # 대기
            time.sleep(2)

            # 스크롤 내린 후 스크롤 높이 다시 가져옴
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
            html = html + driver.page_source
        
        driver.quit()
        soup = BeautifulSoup(html, "html.parser")
        word = soup.find_all("div", attrs={'data-testid': 'tweetText'})
        for w in range(0, len(word)):
            df.loc[n, 'date'] = "{} ~ {}".format(dates[date_id], dates[date_id+1])
            df.loc[n, 'tweet'] = word[w].text 
            n += 1

In [ ]:
# 생성 데이터 확인
df

,date,tweet
0,2022-07-15 ~ 2022-07-16,환승연애2 x커플 모르겠고 이현나연 케미뢰전드.. \n심지어 저웅니들 둘이 룸메임
1,2022-07-15 ~ 2022-07-16,환승연애 최이현 여친짤 재질 미쳤음.. 아름다우셔
2,2022-07-15 ~ 2022-07-16,환승연애 첫선택 #menow
3,2022-07-15 ~ 2022-07-16,환승연애2 최이현 조승우 썰ㅋㅋ
4,2022-07-15 ~ 2022-07-16,환승연애2\n\n해가될까 \n\n#조승연 #WOODZ
...,...,...
5732,2022-11-10 ~ 2022-11-11,이진희도는 환승연애 나가면 안되겠다\n빽도깅이 불 지를지도 모름
5733,2022-11-10 ~ 2022-11-11,@onl_youu _youu 그 환승연애 과몰입에 저도 좀 끼워주세엽
5734,2022-11-10 ~ 2022-11-11,"(ENG) ""해은 제일 잘 나가"" 환승연애 보다가 냅다 케이팝 2시간 동안 갈긴 ..."
5735,2022-11-10 ~ 2022-11-11,다음 댓카에서 환승연애 토론 열어봅시다


In [ ]:
# 데이터 내보내기
df.to_csv('./TW_data.csv',index = False)